# Troubleshoot by Inspecting FileShare Data

In [ ]:
!pip install geopandas

In [ ]:
import azureml.core
from azureml.core import Experiment, Environment, Workspace, Dataset, Datastore, ScriptRunConfig
import os
from os.path import join
import shutil
import numpy as np
import geopandas as gpd

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')
ws.datastores
# connect to our fileshar
datastore = Datastore.get(workspace = ws, datastore_name = 'placefilestore')

## Inspect GeoJson Files

In [ ]:
print(datastore)
root_path = (datastore, 'THC/bastrop_county/data/vector')
share_files = Dataset.File.from_files(path = [root_path])

with share_files.mount() as mount:
    mount_point = mount.mount_point
    pts_path = join(mount_point, 'bastrop_2016.geojson')
    gdf = gpd.read_file(pts_path, driver = 'GeoJSON')

## Inspect numpy files

In [ ]:
root_path = (datastore, 'THC')
share_files = Dataset.File.from_files(path = [root_path])

with share_files.mount() as mount:
    mount_point = mount.mount_point
    npy_files = []
    for root, dirs, files in os.walk(mount_point): 
        for f in files:
            if '.npy' in f and '2020' in root:
                print(root)
                npy_files.append(join(root, f))

    x_files = [f for f in npy_files if 'x_train' in f]
    y_files = [f for f in npy_files if 'y_train' in f]
    xs = [np.load(f, allow_pickle = True) for f in x_files]
    ys = [np.load(f, allow_pickle = True) for f in y_files]
